In [ ]:
#!pip install medmnist

# Imports

In [2]:
import os
import io
import imageio
import medmnist
import ipywidgets
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Setting seed for reproducibility
SEED = 42
os.environ["TF_CUDNN_DETERMINISTIC"] = "1"
keras.utils.set_random_seed(SEED)

2023-05-08 20:09:58.024201: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Hyperparameters

In [3]:
# DATA
DATASET_NAME = "organmnist3d"
BATCH_SIZE = 32
AUTO = tf.data.AUTOTUNE
INPUT_SHAPE = (28, 28, 28, 1)
NUM_CLASSES = 11

# OPTIMIZER
LEARNING_RATE = 1e-4
WEIGHT_DECAY = 1e-5

# TRAINING
EPOCHS = 60

# TUBELET EMBEDDING
PATCH_SIZE = (8, 8, 8)
NUM_PATCHES = (INPUT_SHAPE[0] // PATCH_SIZE[0]) ** 2

# ViViT ARCHITECTURE
LAYER_NORM_EPS = 1e-6
PROJECTION_DIM = 128
NUM_HEADS = 8
NUM_LAYERS = 8

# Dataset

In [4]:
def download_and_prepare_dataset(data_info: dict):
    """Utility function to download the dataset.

    Arguments:
        data_info (dict): Dataset metadata.
    """
    data_path = keras.utils.get_file(origin=data_info["url"], md5_hash=data_info["MD5"])

    with np.load(data_path) as data:
        # Get videos
        train_videos = data["train_images"]
        valid_videos = data["val_images"]
        test_videos = data["test_images"]

        # Get labels
        train_labels = data["train_labels"].flatten()
        valid_labels = data["val_labels"].flatten()
        test_labels = data["test_labels"].flatten()

    return (
        (train_videos, train_labels),
        (valid_videos, valid_labels),
        (test_videos, test_labels),
    )


# Get the metadata of the dataset
info = medmnist.INFO[DATASET_NAME]

import pprint
pprint.pprint(info)

# Get the dataset
prepared_dataset = download_and_prepare_dataset(info)
(train_videos, train_labels) = prepared_dataset[0]
(valid_videos, valid_labels) = prepared_dataset[1]
(test_videos, test_labels) = prepared_dataset[2]

{'MD5': '21f0a239e7f502e6eca33c3fc453c0b6',
 'description': 'The source of the OrganMNIST3D is the same as that of the '
                'Organ{A,C,S}MNIST. Instead of 2D images, we directly use the '
                '3D bounding boxes and process the images into 28×28×28 to '
                'perform multi-class classification of 11 body organs. The '
                'same 115 and 16 CT scans as the Organ{A,C,S}MNIST from the '
                'source training set are used as training and validation set, '
                'respectively, and the same 70 CT scans as the '
                'Organ{A,C,S}MNIST from the source test set are treated as the '
                'test set.',
 'label': {'0': 'liver',
           '1': 'kidney-right',
           '10': 'pancreas',
           '2': 'kidney-left',
           '3': 'femur-right',
           '4': 'femur-left',
           '5': 'bladder',
           '6': 'heart',
           '7': 'lung-right',
           '8': 'lung-left',
           '9': 'spleen

# tf.data pipeline

In [5]:
@tf.function
def preprocess(frames: tf.Tensor, label: tf.Tensor):
    """Preprocess the frames tensors and parse the labels."""
    # Preprocess images
    frames = tf.image.convert_image_dtype(
        frames[
            ..., tf.newaxis
        ],  # The new axis is to help for further processing with Conv3D layers
        tf.float32,
    )
    # Parse label
    label = tf.cast(label, tf.float32)
    return frames, label


def prepare_dataloader(
    videos: np.ndarray,
    labels: np.ndarray,
    loader_type: str = "train",
    batch_size: int = BATCH_SIZE,
):
    """Utility function to prepare the dataloader."""
    dataset = tf.data.Dataset.from_tensor_slices((videos, labels))

    if loader_type == "train":
        dataset = dataset.shuffle(BATCH_SIZE * 2)

    dataloader = (
        dataset.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
        .batch(batch_size)
        .prefetch(tf.data.AUTOTUNE)
    )
    return dataloader


trainloader = prepare_dataloader(train_videos, train_labels, "train")
validloader = prepare_dataloader(valid_videos, valid_labels, "valid")
testloader = prepare_dataloader(test_videos, test_labels, "test")

# Tubelet Embedding

In [6]:
class TubeletEmbedding(layers.Layer):
    def __init__(self, embed_dim, patch_size, **kwargs):
        super().__init__(**kwargs)
        self.projection = layers.Conv3D(
            filters=embed_dim,
            kernel_size=patch_size,
            strides=patch_size,
            padding="VALID",
        )
        self.flatten = layers.Reshape(target_shape=(-1, embed_dim))

    def call(self, videos):
        projected_patches = self.projection(videos)
        flattened_patches = self.flatten(projected_patches)
        return flattened_patches

# Positional Embedding

In [7]:
class PositionalEncoder(layers.Layer):
    def __init__(self, embed_dim, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim

    def build(self, input_shape):
        _, num_tokens, _ = input_shape
        self.position_embedding = layers.Embedding(
            input_dim=num_tokens, output_dim=self.embed_dim
        )
        self.positions = tf.range(start=0, limit=num_tokens, delta=1)

    def call(self, encoded_tokens):
        # Encode the positions and add it to the encoded tokens
        encoded_positions = self.position_embedding(self.positions)
        encoded_tokens = encoded_tokens + encoded_positions
        return encoded_tokens

# Video Vision Transformer

In [8]:
def create_vivit_classifier(
    tubelet_embedder,
    positional_encoder,
    input_shape=INPUT_SHAPE,
    transformer_layers=NUM_LAYERS,
    num_heads=NUM_HEADS,
    embed_dim=PROJECTION_DIM,
    layer_norm_eps=LAYER_NORM_EPS,
    num_classes=NUM_CLASSES,
):
    # Get the input layer
    inputs = layers.Input(shape=input_shape)
    # Create patches.
    patches = tubelet_embedder(inputs)
    # Encode patches.
    encoded_patches = positional_encoder(patches)

    # Create multiple layers of the Transformer block.
    for _ in range(transformer_layers):
        # Layer normalization and MHSA
        x1 = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
        attention_output = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim // num_heads, dropout=0.1
        )(x1, x1)

        # Skip connection
        x2 = layers.Add()([attention_output, encoded_patches])

        # Layer Normalization and MLP
        x3 = layers.LayerNormalization(epsilon=1e-6)(x2)
        x3 = keras.Sequential(
            [
                layers.Dense(units=embed_dim * 4, activation=tf.nn.gelu),
                layers.Dense(units=embed_dim, activation=tf.nn.gelu),
            ]
        )(x3)

        # Skip connection
        encoded_patches = layers.Add()([x3, x2])

    # Layer normalization and Global average pooling.
    representation = layers.LayerNormalization(epsilon=layer_norm_eps)(encoded_patches)
    representation = layers.GlobalAvgPool1D()(representation)

    # Classify outputs.
    outputs = layers.Dense(units=num_classes, activation="softmax")(representation)

    # Create the Keras model.
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model

# Train

In [9]:
def run_experiment():
    # Initialize model
    model = create_vivit_classifier(
        tubelet_embedder=TubeletEmbedding(
            embed_dim=PROJECTION_DIM, patch_size=PATCH_SIZE
        ),
        positional_encoder=PositionalEncoder(embed_dim=PROJECTION_DIM),
    )

    # Compile the model with the optimizer, loss function
    # and the metrics.
    optimizer = keras.optimizers.Adam(learning_rate=LEARNING_RATE)
    model.compile(
        optimizer=optimizer,
        loss="sparse_categorical_crossentropy",
        metrics=[
            keras.metrics.SparseCategoricalAccuracy(name="accuracy"),
            keras.metrics.SparseTopKCategoricalAccuracy(5, name="top-5-accuracy"),
        ],
    )

    # Train the model.
    _ = model.fit(trainloader, epochs=EPOCHS, validation_data=validloader)

    _, accuracy, top_5_accuracy = model.evaluate(testloader)
    print(f"Test accuracy: {round(accuracy * 100, 2)}%")
    print(f"Test top 5 accuracy: {round(top_5_accuracy * 100, 2)}%")

    return model


model = run_experiment()

Epoch 1/60


2023-05-08 20:10:24.102347: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype uint8 and shape [972]
	 [[{{node Placeholder/_1}}]]
2023-05-08 20:10:24.107834: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype uint8 and shape [972,28,28,28]
	 [[{{node Placeholder/_0}}]]


31/31 [==============================] - ETA: 0s - loss: 2.5485 - accuracy: 0.1111 - top-5-accuracy: 0.5597

2023-05-08 20:11:22.924173: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype uint8 and shape [161]
	 [[{{node Placeholder/_1}}]]
2023-05-08 20:11:22.942282: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype uint8 and shape [161]
	 [[{{node Placeholder/_1}}]]


31/31 [==============================] - 73s 1s/step - loss: 2.5485 - accuracy: 0.1111 - top-5-accuracy: 0.5597 - val_loss: 2.3513 - val_accuracy: 0.1988 - val_top-5-accuracy: 0.5776
Epoch 2/60
31/31 [==============================] - 30s 972ms/step - loss: 2.2323 - accuracy: 0.1800 - top-5-accuracy: 0.6687 - val_loss: 2.0809 - val_accuracy: 0.2174 - val_top-5-accuracy: 0.7205
Epoch 3/60
31/31 [==============================] - 22s 710ms/step - loss: 2.0593 - accuracy: 0.2150 - top-5-accuracy: 0.7767 - val_loss: 1.8950 - val_accuracy: 0.3106 - val_top-5-accuracy: 0.8075
Epoch 4/60
31/31 [==============================] - 23s 734ms/step - loss: 1.9996 - accuracy: 0.2263 - top-5-accuracy: 0.7973 - val_loss: 1.7612 - val_accuracy: 0.3230 - val_top-5-accuracy: 0.8509
Epoch 5/60
31/31 [==============================] - 24s 771ms/step - loss: 1.8056 - accuracy: 0.3107 - top-5-accuracy: 0.8663 - val_loss: 1.7071 - val_accuracy: 0.4099 - val_top-5-accuracy: 0.8447
Epoch 6/60
31/31 [===========

31/31 [==============================] - 20s 645ms/step - loss: 0.0382 - accuracy: 0.9907 - top-5-accuracy: 1.0000 - val_loss: 0.4059 - val_accuracy: 0.8944 - val_top-5-accuracy: 0.9938
Epoch 43/60
31/31 [==============================] - 18s 570ms/step - loss: 0.0613 - accuracy: 0.9825 - top-5-accuracy: 1.0000 - val_loss: 0.4556 - val_accuracy: 0.8820 - val_top-5-accuracy: 0.9938
Epoch 44/60
31/31 [==============================] - 19s 609ms/step - loss: 0.0496 - accuracy: 0.9856 - top-5-accuracy: 1.0000 - val_loss: 0.4232 - val_accuracy: 0.9006 - val_top-5-accuracy: 1.0000
Epoch 45/60
31/31 [==============================] - 19s 610ms/step - loss: 0.0234 - accuracy: 0.9928 - top-5-accuracy: 1.0000 - val_loss: 0.4380 - val_accuracy: 0.9006 - val_top-5-accuracy: 0.9876
Epoch 46/60
31/31 [==============================] - 19s 606ms/step - loss: 0.0351 - accuracy: 0.9877 - top-5-accuracy: 1.0000 - val_loss: 0.4302 - val_accuracy: 0.8696 - val_top-5-accuracy: 0.9938
Epoch 47/60
31/31 [===

2023-05-08 20:42:39.305729: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype uint8 and shape [610]
	 [[{{node Placeholder/_1}}]]
2023-05-08 20:42:39.313232: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype uint8 and shape [610]
	 [[{{node Placeholder/_1}}]]


20/20 [==============================] - 3s 151ms/step - loss: 1.0551 - accuracy: 0.7852 - top-5-accuracy: 0.9721
Test accuracy: 78.52%
Test top 5 accuracy: 97.21%


# Inference

In [12]:
NUM_SAMPLES_VIZ = 25
testsamples, labels = next(iter(testloader))
testsamples, labels = testsamples[:NUM_SAMPLES_VIZ], labels[:NUM_SAMPLES_VIZ]

ground_truths = []
preds = []
videos = []

for i, (testsample, label) in enumerate(zip(testsamples, labels)):
    # Generate gif
    with io.BytesIO() as gif:
        imageio.mimsave(gif, (testsample.numpy().squeeze() * 255).astype("uint8"), "GIF", **{'duration': 0.1*1000})
        videos.append(gif.getvalue())

    # Get model prediction
    output = model.predict(tf.expand_dims(testsample, axis=0))[0]
    pred = np.argmax(output, axis=0)

    ground_truths.append(label.numpy().astype("int"))
    preds.append(pred)


def make_box_for_grid(image_widget, fit):
    """Make a VBox to hold caption/image for demonstrating option_fit values.

    Source: https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20Styling.html
    """
    # Make the caption
    if fit is not None:
        fit_str = "'{}'".format(fit)
    else:
        fit_str = str(fit)

    h = ipywidgets.HTML(value="" + str(fit_str) + "")

    # Make the green box with the image widget inside it
    boxb = ipywidgets.widgets.Box()
    boxb.children = [image_widget]

    # Compose into a vertical box
    vb = ipywidgets.widgets.VBox()
    vb.layout.align_items = "center"
    vb.children = [h, boxb]
    return vb


boxes = []
truec = 0
for i in range(NUM_SAMPLES_VIZ):
    ib = ipywidgets.widgets.Image(value=videos[i], width=100, height=100)
    true_class = info["label"][str(ground_truths[i])]
    pred_class = info["label"][str(preds[i])]
    caption = f"T: {true_class} | P: {pred_class}"
    
    if true_class == pred_class :
        truec += 1

    boxes.append(make_box_for_grid(ib, caption))
print(f"{truec}/{NUM_SAMPLES_VIZ}")
ipywidgets.widgets.GridBox(
    boxes, layout=ipywidgets.widgets.Layout(grid_template_columns="repeat(5, 200px)")
)

1/1 [==============================] - 0s 54ms/step
19/25


GridBox(children=(VBox(children=(HTML(value="'T: pancreas | P: pancreas'"), Box(children=(Image(value=b'GIF89a…